In [1]:
import pandas as pd

In [5]:
# mapping of countries names and country codes

country_code_to_name = {
    "at": "Austria", "be": "Belgium", "ca": "Canada", "ch": "Switzerland",
    "cz": "Czech Republic", "de": "Germany", "dk": "Denmark", "es": "Spain",
    "fi": "Finland", "fr": "France", "gb": "United Kingdom", "gr": "Greece",
    "hu": "Hungary", "ie": "Ireland", "is": "Iceland", "it": "Italy",
    "jp": "Japan", "mx": "Mexico", "nl": "Netherlands", "no": "Norway",
    "pl": "Poland", "pt": "Portugal", "se": "Sweden", "sk": "Slovakia",
    "tr": "Turkey", "us": "United States"
}

country_codes = list(country_code_to_name.keys())

country_names = list(country_code_to_name.values())

# columns from original dataset which we find useful for our analysis, due to the size of data we will keep as little data as possible
columns_to_keep = [
    'date', 'region', 'chart', 'streams', 'af_danceability',
    'af_energy', 'af_key', 'af_loudness', 'af_speechiness', 
    'af_acousticness', 'af_valence', 'af_tempo'
]


The function 'read_big_file' and general approach to working on big .csv files was desgined based on the article of Ankush Kunwar from Medium, the link might be found below:

https://ankushkunwar7777.medium.com/working-with-large-csv-file-using-python-1ec6577c5ce6

The 'preprocessing' function itself was created to match our needs during the process of preprocessing. The data is filtered by the date, region, type of charts from which it was aggregated and lastly only the columns which we wanted to keep were selected. 

In [13]:
def preprocessing(data_frame):
    data_frame['date'] = pd.to_datetime(data_frame['date'])
    
    # filter by date
    date_filter = (data_frame['date'] >= '2018-01-01') & (data_frame['date'] <= '2019-12-31')

    # filter by countries names
    region_filter = data_frame['region'].isin(country_names)
    
    data_frame = data_frame[date_filter & region_filter]

    # selection of wanted columns
    data_frame = data_frame[columns_to_keep]

    # filter by chart type
    data_frame = data_frame[data_frame['chart'] == 'top200']

    return data_frame

def read_big_file(file_name):

    for code_chunk in pd.read_csv(file_name, chunksize = 10000):
        yield code_chunk


Below loop allows to create the 'processed_data.csv' which contains the data from original source after our preprocessing. The loop was upgraded from appending empty chunks to simply continuing to save processing time. After the first chunk, where the headers are written to the file, the data is simply appended to the existing file if the next chunk is not empty.

In [14]:
output_file = 'processed_data.csv'

header_written = False

for data_frame in read_big_file('merged_data.csv'):
    processed_data = preprocessing(data_frame)

    if processed_data.empty:
        continue

    if not header_written:
        processed_data.to_csv(output_file, mode='w', index=False)
        header_written = True
    else:
        processed_data.to_csv(output_file, mode='a', header=False, index=False)


In [17]:
processed_df = pd.read_csv('processed_data.csv')

# qualitative check if we have the same number of countries - its OK, 26 countries
len(processed_df['region'].unique())

26

Final preparation of aggregated by month and country data. Dynamic dictionary comprehension is used so that we can only change contents of 'columns_to_average' if any modifications are needed.
The final data frame is 'aggregated_monthly_data.csv' and it will be used in further analysis and modelling.

In [18]:
# making sure that the dates are in correct format
processed_df['date'] = pd.to_datetime(processed_df['date'])

# changning date format so that it matches original data
processed_df['year_month'] = processed_df['date'].dt.strftime('%Y%m')

# chosen columns on which the aggregation will be averaged for the month and region
columns_to_average = [
    'af_danceability', 'af_energy', 'af_key', 'af_loudness', 
    'af_speechiness', 'af_acousticness', 'af_valence', 'af_tempo'
]

# group by 'region' and 'year_month' to calculate aggregate values
# .reset_index to get region and year_month back to be columns, not multi-indexes
aggregated_df = processed_df.groupby(['region', 'year_month']).agg({
    'streams': 'sum',
    **{col: 'mean' for col in columns_to_average}
}).reset_index()

aggregated_df.to_csv('aggregated_monthly_data.csv', index=False)